# Environment setup

In [ ]:
!pip install -q -U transformers peft accelerate optimum
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/
!pip install langchain
!pip install einops
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu117/


In [ ]:
!pip install accelerate

In [ ]:
!pip install huggingface-cli
!pip install evaluate
!pip install langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.3 MB/s eta 0:00:00


# Environment settings

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 0
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:

!pip install flask flask-restful
!pip install PyPDF2
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.6 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/

## Import neccessary libraries

In [ ]:
import time

import numpy as np
import math
import evaluate
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain import LLMChain, HuggingFacePipeline, PromptTemplate

In [ ]:
import accelerate

## Configurations

In [ ]:
config={
    "model_id": ["TheBloke/Llama-2-7b-Chat-GPTQ",],
    "hf_token": "...",
    "model": {
        "temperature": 0.7, # [0, 0.7, .0.9, 1.1, 1.3]  Testing iteratively.
        "max_length": 5000,
        "top_k": 10,
        "num_return": 1
    },
    "dataset": [
        "billsum",
        "cnn_dailymail",
        "big_patent"
    ],
    "eval": ['rouge', "bertscore", 'meteor']
}


# Utils

In [ ]:
def call_parameter(model):
  pytorch_total_params = sum(p.numel() for p in model.parameters())
  trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  untrainable_params = pytorch_total_params - trainable_params
  print(f'Model {model.__class__.__name__} has {pytorch_total_params} parameters in total\n'\
        f'Trainable parameters: {trainable_params}\nUntrainable parameters: {untrainable_params}')
  return pytorch_total_params

# Dataset

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
billsum['test']

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 248
})

# Model architecture

In [ ]:
def generate_model(model_id, config):
  print(f"Setting up model {model_id}")
  model = AutoModelForCausalLM.from_pretrained(model_id, use_safetensors=True,
                            device_map='auto', trust_remote_code=True)
  tokenizer = AutoTokenizer.from_pretrained(model_id,
                                            device_map='auto', trust_remote_code=True)
  return model, tokenizer

In [ ]:
class Agent:
  def __init__(self, model, tokenizer):
    self.model = model
    self.tokenizer = tokenizer

  def __repr__(self):
    return f'Model {self.model.__class__.__name__}'

In [ ]:
#models = []
#for model in config['model_id']:
 #  try:
  #     model, tokenizer = generate_model(model, config)
   #    no_params = call_parameter(model)
    #   print("====="*5)
     #  print(f'Model {model.name} has {no_params} parameters.')
      # agent = Agent(model, tokenizer)
       #models.append(agent)
 #  except:
  #   continue

In [ ]:
model, tokenizer = generate_model(config['model_id'][0], config)
no_params = call_parameter(model)
print("====="*5)
print(f"Model {config['model_id'][0]} has {no_params} parameters.")

Setting up model TheBloke/Llama-2-7b-Chat-GPTQ


config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Model LlamaForCausalLM has 262410240 parameters in total
Trainable parameters: 262410240
Untrainable parameters: 0
Model TheBloke/Llama-2-7b-Chat-GPTQ has 262410240 parameters.


# Experiments

## Implementation

In [ ]:
class Generator:
  def __init__(self, config, agent, template):
    self.agent = agent
    pipeline = transformers.pipeline(
        "text-generation",
        model=self.agent.model,
        tokenizer=self.agent.tokenizer,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto",
        max_length=config['model']['max_length'],
        do_sample=True,
        top_k=config['model']['top_k'],
        num_return_sequences=config['model']['num_return'],
        pad_token_id=tokenizer.eos_token_id
    )
    llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature': config['model']['temperature']})
    prompt = PromptTemplate(template=template, input_variables=["text"])
    self.llm_chain = LLMChain(prompt=prompt, llm=llm)

  def generate(self, text):
    result = self.llm_chain.invoke(text)
    return result

In [ ]:
template = """
              Write a summary of the following text delimited by triple backticks.
              Return your response which covers the key points of the text.
              ```{text}```
              SUMMARY:
           """

In [ ]:
agent = Agent(model, tokenizer)
llm_agent = Generator(config, agent, template)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
sample = billsum['test'][0]
doc = '\n'.join([sample['title'], sample['text']])
generated_sample = llm_agent.generate(doc)
generated_sample

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


{'text': "\n              Write a summary of the following text delimited by triple backticks.\n              Return your response which covers the key points of the text.\n              ```An act to amend Section 44253.4 of the Education Code, relating to teacher credentialing.\nThe people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 44253.4 of the Education Code is amended to read:\n44253.4.\n(a) The commission shall issue an authorization for a teacher to provide all of the following services to limited-English-proficient pupils:\n(1) Instruction for English language development in preschool, kindergarten, grades 1 to 12, inclusive, and classes organized primarily for adults, except when the requirement specified in paragraph (1) of subdivision (b) is satisfied by the possession of a children’s center instructional permit pursuant to Sections 8363 and 44252.7, a children’s center supervision permit pursuant to Section 8363, or a designated subjects teachi

In [ ]:
sample['summary']

'Existing law requires the Commission on Teacher Credentialing to issue authorizations for a teacher to provide specific services to limited-English-proficient pupils, if certain minimum requirements are met.\nThis bill would provide that a teacher who possesses any of several specified California credentials or permits, and who is able to present a valid out-of-state credential or certificate that authorizes content instruction delivered in a pupil’s primary language, may qualify for that authorization by submitting an application and a fee to the commission.'

In [ ]:
text = """
A recent study published in the prestigious journal Nature has shed light on a potential connection between the trillions of bacteria residing in our gut and our mental well-being. Researchers conducted a fascinating experiment involving mice. They observed that mice with disrupted gut microbiomes, the complex community of bacteria within their digestive tract, exhibited behaviors indicative of anxiety and depression. Interestingly, when these mice received fecal transplants from healthy donors with balanced gut bacteria, their symptoms improved significantly. This groundbreaking research suggests that the gut microbiome may play a crucial role in regulating mood and behavior.

Scientists believe the gut-brain connection is facilitated by the intricate network of nerves and hormones that connect our digestive system to the central nervous system. The gut bacteria produce various metabolites, including short-chain fatty acids, that can influence the production of neurotransmitters like serotonin, which plays a key role in mood regulation. Further research is needed to fully understand the mechanisms at play, but this study opens exciting avenues for exploring the potential of manipulating gut bacteria to treat mental health disorders.
"""

In [ ]:
llm_agent.generate(text)

{'text': '\n              Write a summary of the following text delimited by triple backticks.\n              Return your response which covers the key points of the text.\n              ```\nA recent study published in the prestigious journal Nature has shed light on a potential connection between the trillions of bacteria residing in our gut and our mental well-being. Researchers conducted a fascinating experiment involving mice. They observed that mice with disrupted gut microbiomes, the complex community of bacteria within their digestive tract, exhibited behaviors indicative of anxiety and depression. Interestingly, when these mice received fecal transplants from healthy donors with balanced gut bacteria, their symptoms improved significantly. This groundbreaking research suggests that the gut microbiome may play a crucial role in regulating mood and behavior.\n\nScientists believe the gut-brain connection is facilitated by the intricate network of nerves and hormones that connect

# Application

## From real-life document

In [ ]:
llm_agent.generate(text)

KeyboardInterrupt: 

Flask

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/data/diploma

/content/drive/MyDrive/Colab Notebooks/data/diploma


In [ ]:
!pip install flask flask-restful
!pip install PyPDF2
!pip install pyngrok


In [ ]:
from flask_restful import Api, Resource
from flask import Flask, render_template, request, jsonify
from werkzeug.utils import secure_filename
import os
import PyPDF2
import torch
import re
from pyngrok import ngrok


ngrok.set_auth_token("2h3p7rU3aAwS0rvmEfRCmiZ2hMQ_4mmX4nPPAJQfag9q1rw4c")


In [ ]:
app = Flask(__name__)
api = Api(app)


In [ ]:

UPLOAD_FOLDER = 'uploads'
STATIC_FOLDER = 'static'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app = Flask(__name__, static_folder=STATIC_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text
@app.route('/')
def index_home():
    return render_template('index2.html')

@app.route('/summarize', methods=['POST'])
def summarize_text():
    try:
        if 'file' in request.files:
            file = request.files['file']
            if file.filename == '':
                return 'No selected file'

            # Save uploaded file
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)

            # Extract text from PDF file
            text = extract_text_from_pdf(file_path)

            # Load model and tokenizer
            model_id = config['model_id'][0]  # Use the first model in the list
            model, tokenizer = generate_model(model_id, config)
            agent = Agent(model, tokenizer)
            template = """
            Write a summary of the following text delimited by triple backticks.
            Return your response which covers the key points of the text.
            ```{text}```
            SUMMARY:
            """
            llm_agent = Generator(config, agent, template)
            summary = llm_agent.generate(text)

            # Delete the uploaded file after processing
            os.remove(file_path)

            # Return the summary as a response
            summary_str = str(summary)
            return jsonify({'summary': summary_str})

        else:
            data = request.get_json()
            text = data.get('input_text')
            # Load model and tokenizer
            model_id = config['model_id'][0]
            model, tokenizer = generate_model(model_id, config)
            agent = Agent(model, tokenizer)
            template = """
            Write a summary of the following text delimited by triple backticks.
            Return your response which covers the key points of the text.
            ```{text}```
            SUMMARY:
            """
            llm_agent = Generator(config, agent, template)
            summary = llm_agent.generate(text[:config['model']['max_length']])  # Truncate text to max_length
            return jsonify({'summary': summary})

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return str(e), 500

# Create the ngrok tunnel
url = ngrok.connect(5000).public_url
print(' * Tunnel URL:', url)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Tunnel URL: https://1231-35-240-237-186.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 18:52:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 18:52:01] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 18:52:01] "GET /static/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 18:52:02] "GET /favicon.ico HTTP/1.1" 404 -


Setting up model TheBloke/Llama-2-7b-Chat-GPTQ


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp

# Reference
+ https://colab.research.google.com/drive/1_TIrmuKOFhuRRiTWN94iLKUFu6ZX4ceb?usp=sharing#scrollTo=pDU1jZvWp7s5
+ https://huggingface.co/spaces/evaluate-metric/rouge
+ https://huggingface.co/spaces/evaluate-metric/bertscore
+ https://huggingface.co/spaces/evaluate-metric/meteor